In [ ]:
import warnings
import numpy as np
from scipy.stats import pearsonr
from nilearn.image import load_img
from nilearn.maskers import NiftiSpheresMasker
from brainvoyagertools.voi import VOIsDefinition as voi_reader
import matplotlib.pyplot as plt
%matplotlib widget

Ignore some warnnigs about NaNs and Infs

In [ ]:
warnings.filterwarnings("ignore")

Read nii & VOI file

In [ ]:
nii = load_img('../fMRI Data for Tutorial and HW/sub-025-task-intact2.nii')
voi = voi_reader(load='../fMRI Data for Tutorial and HW/a1_group2_new.voi')

Create Masks from VOIs

In [ ]:
nii_voitc = {}
for single_voi in voi.vois:
    masker = NiftiSpheresMasker(seeds=single_voi.data)
    nii_voitc[single_voi.name] = masker.fit_transform(nii)

In [ ]:
nii_avg_voitc = {}
for key in nii_voitc:
    nii_avg_voitc[key] = np.mean(nii_voitc[key], 1)

plot average timecourse

In [ ]:
fig = plt.figure(figsize=(7, 10))
for num, key in enumerate(nii_avg_voitc):
    ax = fig.add_subplot(len(nii_avg_voitc), 1, num+1)
    ax.plot(nii_avg_voitc[key])
    ax.set_title(f'Region Of Intrest {key}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Magnitude')
plt.tight_layout()
plt.show()

Correlation between average brain area

In [ ]:
n = len(nii_avg_voitc)  # number of brain area
R = np.zeros((n, n))

str_mat = []
for indx, key_1 in enumerate(nii_avg_voitc):
    for jndx, key_2 in enumerate(nii_avg_voitc):
        val, _ = pearsonr(nii_avg_voitc[key_1], nii_avg_voitc[key_2])
        R[indx, jndx] = val

Plot Correlation Matrix

In [ ]:
fig, ax = plt.subplots()
ax_img = ax.imshow(R, extent=[-1, 1, -1, 1])
fig.colorbar(ax_img)
ax.set_xticks(ticks=np.linspace(-0.75, 0.75, 5), labels=voi.names)
ax.set_yticks(ticks=np.linspace(-0.75, 0.75, 5), labels=voi.names[::-1])
plt.show()